In [292]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 


Extraction des fichier csv 

In [293]:
train__set_features = pd.read_csv("training_set_features.csv")
test = pd.read_csv("test_set_features.csv")
train__set_features.shape

train__set_labels = pd.read_csv("training_set_labels.csv")

y_h1n1 = train__set_labels['h1n1_vaccine']
y_seasonal = train__set_labels['seasonal_vaccine']

train__set_features.head()



,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb


In [294]:
train = pd.concat([train__set_features, y_h1n1, y_seasonal], axis=1)


On va gerer les valeurs manquantes et les valeurs categoriels 

In [295]:
train.select_dtypes(include='object').isna().sum()


# la moitié de employement _industry et _occupation est manquante 
train['employment_occupation'].value_counts()

# essayer de creer catagorie big industrie // small industrie 



def employment_cat(X):
    small_jobs = train['employment_occupation'].value_counts() < 500
    small_industries = train['employment_industry'].value_counts() < 500
    X.loc[X['employment_occupation'].isin(small_jobs.index[small_jobs]), 'employment_occupation'] = 'small_job'
    X.loc[X['employment_industry'].isin(small_industries.index[small_industries]), 'employment_industry'] = 'small_industry'
    return X 

train = employment_cat(train)
test = employment_cat(test)

print(test.shape, train.shape)

(26708, 36) (26707, 38)


In [296]:
col = train.select_dtypes(exclude=(np.number)).columns

train = pd.get_dummies(train, columns=col)
test = pd.get_dummies(test, columns=col)

print(test.shape, train.shape)

(26708, 106) (26707, 84)


Creation des colonnes dans le cas ou la categorie n'était pas le test set 

In [297]:
print(test.shape, train.shape)
for col in train.columns: 
    if not(col in test.columns) and not(col in ['seasonal_vaccine', 'h1n1_vaccine']):
        test[col] = pd.Series(np.zeros(test.shape[0]), name=col)
print(test.shape, train.shape)

for col in test.columns:
    if not(col in train.columns):
        train[col] = pd.Series(np.zeros(train.shape[0]), name=col)
print(test.shape, train.shape)


(26708, 106) (26707, 84)
(26708, 108) (26707, 84)
(26708, 108) (26707, 110)


Estimation des valeurs manquantes grace aux KNN 


In [298]:

# essayer d'utiliser le test set pour les NA
 
from sklearn.neighbors import KNeighborsRegressor

def manage_na(X,na_col): 
    X = X.copy()
    numerical = X.select_dtypes(np.number)
    non_na_columns = numerical.loc[:,numerical.isna().sum() == 0].columns
    X_train = numerical.loc[numerical[na_col].isna() == False,non_na_columns]
    y_train = numerical.loc[numerical[na_col].isna() == False,na_col]
    X_na = numerical.loc[numerical[na_col].isna() == True, non_na_columns]
    knn = KNeighborsRegressor()
    knn.fit(X_train, y_train)
    y_na = knn.predict(X_na)
    X.loc[X[na_col].isna() == True, na_col] = y_na
    
    return X 

In [299]:
cond = train.corr()['seasonal_vaccine'].apply(abs).sort_values() > 0.09


print(train.loc[:, cond_h1n1 == True].columns)
print(train.loc[:, cond == True].columns)



nulls = train[cond.index[cond]].isna().sum() > 0


for col in nulls[nulls == True].index:
    train = manage_na(train, col)

nulls_test = test.isna().sum() > 0

for col in nulls_test[nulls_test == True].index:
    test = manage_na(test, col)

corr = train.corr()['seasonal_vaccine'].apply(abs).sort_values()


Index(['h1n1_concern', 'h1n1_knowledge', 'doctor_recc_h1n1',
       'doctor_recc_seasonal', 'chronic_med_condition', 'health_worker',
       'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
       'opinion_seas_vacc_effective', 'opinion_seas_risk', 'h1n1_vaccine',
       'seasonal_vaccine', 'employment_industry_fcxhlnwr',
       'employment_occupation_cmhcxjea'],
      dtype='object')
Index(['h1n1_concern', 'h1n1_knowledge', 'behavioral_wash_hands',
       'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal',
       'chronic_med_condition', 'health_worker', 'health_insurance',
       'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
       'opinion_seas_vacc_effective', 'opinion_seas_risk',
       'household_children', 'h1n1_vaccine', 'seasonal_vaccine',
       'age_group_18 - 34 Years', 'age_group_65+ Years', 'race_White',
       'rent_or_own_Own', 'rent_or_own_Rent',
       'employment_status_Not in Labor Force', 'employment_industry_fcxhlnwr',
       'employment_

In [300]:
nulls = train.isna().sum().sort_values() > 0


In [301]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split


X = train


params = {
    "n_estimators": np.arange(7, 10,1), 
    "max_depth": np.arange(7, 10, 1),
    "min_samples_leaf": np.arange(12,15 , 1), 
}
grid = GridSearchCV(RandomForestRegressor(), params, cv=5)

nulls = X.isna().sum() > 0 

for col in nulls[nulls == True].index:
    X = manage_na(X, col)


X = X.drop(["h1n1_vaccine", "seasonal_vaccine"], axis=1).sort_index(axis=1)
test = test.sort_index(axis=1)

X_train, X_test, y_train, y_test_seas = train_test_split(X, y_seasonal,  test_size=0.2, random_state=32,)

#grid.fit(X_train, y_train)
# grid.best_params_


In [303]:
import tensorflow as tf 

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(108, activation="relu"))
model.add(tf.keras.layers.Dense(54, activation="relu"))
model.add(tf.keras.layers.Dropout(.3))
model.add(tf.keras.layers.Dense(1, activation="relu"))

model.compile(optimizer="adam", loss="mse")

model.fit(X_train, y_train, epochs=200)



Epoch 1/200
668/668 [==============================] - 4s 4ms/step - loss: 65.2112
Epoch 2/200
668/668 [==============================] - 3s 4ms/step - loss: 0.4828
Epoch 3/200
668/668 [==============================] - 3s 5ms/step - loss: 15.0397
Epoch 4/200
668/668 [==============================] - 3s 5ms/step - loss: 0.4658
Epoch 5/200
668/668 [==============================] - 3s 5ms/step - loss: 0.4658
Epoch 6/200
668/668 [==============================] - 3s 5ms/step - loss: 0.4658
Epoch 7/200
668/668 [==============================] - 3s 4ms/step - loss: 0.4658
Epoch 8/200
668/668 [==============================] - 3s 4ms/step - loss: 0.4658
Epoch 9/200
668/668 [==============================] - 3s 5ms/step - loss: 0.4658
Epoch 10/200
668/668 [==============================] - 3s 5ms/step - loss: 0.4658
Epoch 11/200
  1/668 [..............................] - ETA: 10s - loss: 0.4062

KeyboardInterrupt: 

In [ ]:
# rf = RandomForestRegressor(max_depth=9,min_samples_leaf=13, n_estimators=9)

# rf.fit(X_train, y_train)

y_seasonal_predict = model.predict(test)
y_seasonal_predict_test = model.predict(X_test)

# rf.score(X_test, y_test_seas)
y_seasonal_predict.mean()
y_seasonal_predict.shape


(26708,)

In [ ]:
X_train, X_test, y_train, y_test_h1n1 = train_test_split(X, y_h1n1,  test_size=0.2, random_state=32,)

rf = RandomForestRegressor(max_depth=9,min_samples_leaf=12, n_estimators=9)

rf.fit(X_train, y_train)

#grid.fit(X_train, y_train)
#print(grid.best_params_)



y_h1n1_predict = rf.predict(test)
y_h1n1_predict_test = rf.predict(X_test)


print(rf.score(X_test, y_test_h1n1))
y_h1n1_predict.mean()

result = pd.concat([test["respondent_id"], pd.Series(y_h1n1_predict, name="h1n1_vaccine"), pd.Series(y_seasonal_predict, name="seasonal_vaccine")], axis=1)
result.to_csv("result_employment.csv", index=False)




0.2812587818512391


In [ ]:
sub_format = pd.read_csv("submission_format.csv")


In [ ]:
from sklearn.metrics import roc_auc_score



y_true = np.hstack([y_test_h1n1, y_test_seas])
y_score = np.hstack([y_h1n1_predict_test, y_seasonal_predict_test])


roc_auc_score(y_true, y_score)


0.8581082868733994